In [1]:
import os
import sys
import random
import torch
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR

In [2]:
sys.path.append("..")

In [3]:
os.cpu_count()

64

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.device_count()

4

In [6]:
from carca.data import CARCADataset, load_attrs, load_ctx, load_profiles
from carca.model import CARCA
from carca.train import train, evaluate

In [7]:
attrs = load_attrs("video_games")
ctx = load_ctx("video_games")
user_ids, item_ids, profiles = load_profiles("video_games")

In [8]:
n_items = len(item_ids) + 1
n_ctx = next(iter(ctx.values())).shape[0]
n_attrs = attrs.shape[1]

In [9]:
# Hyper-parameters for Games dataset
learning_rate =  0.0001
seq_len = 50
n_blocks = 3
n_heads = 3
dropout_rate = 0.5
l2_reg = 0.0
d_dim = 90
g_dim = 450
residual_sa = True
residual_ca = True
epochs = 800
batch_size = 128
beta1 = 0.9
beta2 = 0.98

In [10]:
train_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="train"
)
val_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="val"
)
test_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="test"
)

val_idx = random.sample(range(len(val_data)), 10_000) if len(val_data) > 10_000 else range(len(val_data))
val_sub = Subset(val_data, val_idx)
test_idx = random.sample(range(len(test_data)), 10_000) if len(test_data) > 10_000 else range(len(test_data))
test_sub = Subset(test_data, test_idx)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_sub, batch_size=batch_size, shuffle=False, num_workers=8)
test_loader = DataLoader(test_sub, batch_size=batch_size, shuffle=False, num_workers=8)

In [11]:
model = CARCA(
    n_items=n_items,
    d=d_dim,
    g=g_dim,
    n_ctx=n_ctx,
    n_attrs=n_attrs,
    H=n_heads,
    p=dropout_rate,
    B=n_blocks,
    res_sa=residual_sa,
    res_ca=residual_ca
)

In [12]:
optim = Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg, betas=(beta1, beta2))

In [13]:
# optim = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=l2_reg)
# scheduler = CosineAnnealingLR(optim, epochs, verbose=True)

In [14]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda:2"
print(f"Using {device} device")
model = model.to(device)

Using cuda:2 device


In [15]:
model = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    optim=optim,
    epochs=epochs,
    early_stop=100,
    # scheduler=scheduler
)

16:22:07 - Epoch 001: Loss = 0.6923
16:22:08 - Epoch 001: Loss = 0.7117 HR = 0.1022, NDCG = 0.0460
16:22:16 - Epoch 002: Loss = 0.6906
16:22:17 - Epoch 002: Loss = 0.6844 HR = 0.1096, NDCG = 0.0508
16:22:24 - Epoch 003: Loss = 0.6834
16:22:26 - Epoch 003: Loss = 0.6837 HR = 0.1190, NDCG = 0.0546
16:22:33 - Epoch 004: Loss = 0.6599
16:22:35 - Epoch 004: Loss = 0.6438 HR = 0.2178, NDCG = 0.0948
16:22:42 - Epoch 005: Loss = 0.6223
16:22:44 - Epoch 005: Loss = 0.6630 HR = 0.2821, NDCG = 0.1360
16:22:51 - Epoch 006: Loss = 0.6051
16:22:53 - Epoch 006: Loss = 0.6450 HR = 0.2900, NDCG = 0.1459
16:23:00 - Epoch 007: Loss = 0.5958
16:23:02 - Epoch 007: Loss = 0.6501 HR = 0.3110, NDCG = 0.1577
16:23:09 - Epoch 008: Loss = 0.5870
16:23:10 - Epoch 008: Loss = 0.6217 HR = 0.3307, NDCG = 0.1724
16:23:18 - Epoch 009: Loss = 0.5717
16:23:19 - Epoch 009: Loss = 0.6181 HR = 0.3553, NDCG = 0.2014
16:23:26 - Epoch 010: Loss = 0.5485
16:23:28 - Epoch 010: Loss = 0.5403 HR = 0.4313, NDCG = 0.2526
16:23:35 -

In [16]:
model = torch.load("model/210_0.7424.pth")

In [24]:
torch.set_printoptions(linewidth=500, edgeitems=10, precision=4, sci_mode=False)

In [17]:
evaluate(model, test_loader, device, 10)

(0.7246, 0.48835986981391905, 0.189534300678893)